# Librerias

In [23]:
import pandas as pd
import utils as u
from datetime import datetime, timedelta

# URL 1: Alta de usuarios y usos por día del servicio público de bicicleta eléctrica

### Alta Usuarios

Limpieza de las bases de usuarios: datos historicos de usuario con sus altas y bajas.

In [24]:
# Lectura de csv

usuario_abonos_anual = pd.read_excel('../data/Datos Originales/bicis_usuarios_abonos_202210.xlsx', sheet_name='Abono anual activos')
usuario_abonos_ocacional = pd.read_excel('../data/Datos Originales/bicis_usuarios_abonos_202210.xlsx', sheet_name='Abono ocasional datos')

In [25]:
#Usuarios Abonos Anual --> Limpieza de NAN
usuario_abonos_anual.dropna(axis=0, how='all', inplace=True)
usuario_abonos_anual.dropna(axis=1, how='all', inplace=True)

#Usuarios Abonos Ocacional --> Limpieza de NAN
usuario_abonos_ocacional.dropna(axis=0, how='all', inplace=True)
usuario_abonos_ocacional.dropna(axis=1, how='all', inplace=True)

Para estos datos vamos a enforcarnos solo en el año 2022, filtamos los Dataframe para el año.

In [26]:
# Crear df para el 2022

usuario_abonos_anual_2022 = usuario_abonos_anual[usuario_abonos_anual['Año']==2022].reset_index(drop=True)
usuario_abonos_ocacional_2022 = usuario_abonos_ocacional[usuario_abonos_ocacional['Año']==2022].reset_index(drop=True)

In [27]:
# Filtramos las columnas que nos interesan y las renombramos para que tengan un nombre más amigable en los abonos anuales.

usuario_abonos_anual_2022 = usuario_abonos_anual_2022[['Año', 
                                                       'Mes', 
                                                       'Día', 
                                                       'Altas nuevos usuarios activos ab. anual total día', 
                                                       'Altas nuevos usuarios ab. anual total acumulado mes',
                                                       'Usuarios activos ab. anual total acumulado desde inicio']]

usuario_abonos_anual_2022.rename(columns={'Altas nuevos usuarios activos ab. anual total día': 'alta_ab_anual_dia',
                                        'Altas nuevos usuarios ab. anual total acumulado mes': 'alta_ab_anual_mes',
                                        'Año':'anio',
                                        'Día':'fecha',
                                        'Usuarios activos ab. anual total acumulado desde inicio':'ab_anual_total'}, inplace=True)

In [28]:
# Filtramos las columnas que nos interesan y las renombramos para que tengan un nombre más amigable en los abonos ocasionales.

usuario_abonos_ocacional_2022 = usuario_abonos_ocacional_2022[['Año', 
                                                                'Mes', 
                                                                'Día', 
                                                                'Nuevos usuarios abono 1 día ', 
                                                                'Nuevos usuarios abono 3 días ',
                                                                'Nuevos usuarios abono 5 días ',
                                                                'Nuevos usuarios abono ocas. total',
                                                                'Usuarios abono 1 día acumulado mes',
                                                                'Usuarios abono 3 días acumulado mes',
                                                                'Usuarios abono 5 días acumulado mes',
                                                                'Usuarios abono ocasional total mes',
                                                                'Usuarios abono 1 día acumulado de inicio',
                                                                'Usuarios abono 3 días acumulado de inicio',
                                                                'Usuarios abono 5 días acumulado de inicio',
                                                                'Usuarios abono ocasional total de inicio']]

usuario_abonos_ocacional_2022.rename(columns={'Nuevos usuarios abono 1 día ': 'alta_ab_1d_dia',
                                        'Nuevos usuarios abono 3 días ': 'alta_ab_3d_dia',
                                        'Nuevos usuarios abono 5 días ': 'alta_ab_5d_dia',
                                        'Nuevos usuarios abono ocas. total': 'alta_ab_ocas_dia',
                                        'Usuarios abono 1 día acumulado mes': 'alta_ab_1d_mes',
                                        'Usuarios abono 3 días acumulado mes': 'alta_ab_3d_mes',
                                        'Usuarios abono 5 días acumulado mes': 'alta_ab_5d_mes',
                                        'Usuarios abono ocasional total mes': 'alta_ab_ocas_mes',
                                        'Usuarios abono 1 día acumulado de inicio': 'ab_1d_total',
                                        'Usuarios abono 3 días acumulado de inicio': 'ab_3d_total',
                                        'Usuarios abono 5 días acumulado de inicio': 'ab_5d_total',
                                        'Usuarios abono ocasional total de inicio': 'ab_ocas_total',
                                        'Año':'anio',
                                        'Día':'fecha'}, inplace=True)

Una vez limpias las bases vamos a unificar todo en un dataframe.

In [29]:
''' 
Union de las dos fuentes por la fecha, nos quedamos solos con los campos de fecha de usuario_abonos_anual_2022

- Quitamos columnas duplicadas por el merge.
- Renombramos columnas.

'''

usuarios = pd.merge(usuario_abonos_anual_2022, usuario_abonos_ocacional_2022, on='fecha')

usuarios.drop(['anio_y', 'Mes_y'], axis=1, inplace=True)

usuarios.rename(columns={'anio_x':'anio',
                         'Mes_x': 'mes'}, inplace=True)

Una vez unificadas la fuentes vamos a evaluar el tipo de dato y si tienen nulos o no.

In [30]:
# Ninguna variable tienen nulls

usuarios.isnull().any()

anio                 False
mes                  False
fecha                False
alta_ab_anual_dia    False
alta_ab_anual_mes    False
ab_anual_total       False
alta_ab_1d_dia       False
alta_ab_3d_dia       False
alta_ab_5d_dia       False
alta_ab_ocas_dia     False
alta_ab_1d_mes       False
alta_ab_3d_mes       False
alta_ab_5d_mes       False
alta_ab_ocas_mes     False
ab_1d_total          False
ab_3d_total          False
ab_5d_total          False
ab_ocas_total        False
dtype: bool

In [31]:
''' 
Para nuestra analítica creamos columnas de fechas para poder trabajabar sobre todo la estacionalidad.
- Día de la semana: El número de día sabiendo que 0 es domingo y 6 es sabado.
- Fin de Semana: Variable binaria que indica si el día es sabado o domingo.

'''

usuarios['dia_semana'] = [int(str(datetime.strftime(i, '%w'))) for i in usuarios['fecha']]
usuarios['fin_semana'] = [1 if i in [0, 6] else 0 for i in usuarios['dia_semana']]

In [32]:
# Reordenamos las columnas 

usuarios = usuarios.loc[:, ['anio', 
                            'mes', 
                            'fecha',
                            'dia_semana',
                            'fin_semana', 
                            'alta_ab_anual_dia', 
                            'alta_ab_anual_mes',
                            'alta_ab_1d_dia', 
                            'alta_ab_3d_dia', 
                            'alta_ab_5d_dia',
                            'alta_ab_ocas_dia', 
                            'alta_ab_1d_mes', 
                            'alta_ab_3d_mes',
                            'alta_ab_5d_mes', 
                            'alta_ab_ocas_mes',
                            'ab_anual_total',
                            'ab_1d_total',
                            'ab_3d_total',
                            'ab_5d_total',
                            'ab_ocas_total']]

In [33]:
# Convertimos en enteros las variables desde la columna 5 al final.

for i in usuarios.iloc[:, 5:]:
    usuarios[i] = [int(x) for x in usuarios[i]]

In [34]:
# Cambios de tipos de variables

usuarios.dtypes

anio                          int64
mes                           int64
fecha                datetime64[ns]
dia_semana                    int64
fin_semana                    int64
alta_ab_anual_dia             int64
alta_ab_anual_mes             int64
alta_ab_1d_dia                int64
alta_ab_3d_dia                int64
alta_ab_5d_dia                int64
alta_ab_ocas_dia              int64
alta_ab_1d_mes                int64
alta_ab_3d_mes                int64
alta_ab_5d_mes                int64
alta_ab_ocas_mes              int64
ab_anual_total                int64
ab_1d_total                   int64
ab_3d_total                   int64
ab_5d_total                   int64
ab_ocas_total                 int64
dtype: object

In [35]:
usuarios.head()

,anio,mes,fecha,dia_semana,fin_semana,alta_ab_anual_dia,alta_ab_anual_mes,alta_ab_1d_dia,alta_ab_3d_dia,alta_ab_5d_dia,alta_ab_ocas_dia,alta_ab_1d_mes,alta_ab_3d_mes,alta_ab_5d_mes,alta_ab_ocas_mes,ab_anual_total,ab_1d_total,ab_3d_total,ab_5d_total,ab_ocas_total
0,2022,1,2022-01-01,6,1,-16,-16,36,4,1,41,36,4,1,41,67986,147719,14062,6522,168303
1,2022,1,2022-01-02,0,1,19,3,22,1,0,23,58,5,1,64,68005,147741,14063,6522,168326
2,2022,1,2022-01-03,1,0,15,18,12,1,2,15,70,6,3,79,68020,147753,14064,6524,168341
3,2022,1,2022-01-04,2,0,-27,-9,2,0,0,2,72,6,3,81,67993,147755,14064,6524,168343
4,2022,1,2022-01-05,3,0,-37,-46,3,0,0,3,75,6,3,84,67956,147758,14064,6524,168346


In [36]:
# Base de datos tratado de usuario en el 2022.

usuarios.to_csv('../data/Datos Tratados/usuarios_bici_resumen_2022.csv', index=False)

### Uso por dia

In [37]:
# Lectura de csv

uso_bicicletas = pd.read_excel('../data/Datos Originales/bicis_usos_acumulado_202210.xlsx', sheet_name='Tabla usos')

In [38]:
# Uso Abonos --> Limpieza de NAN

uso_bicicletas.dropna(axis=0, how='all', inplace=True)
uso_bicicletas.dropna(axis=1, how='all', inplace=True)
uso_bicicletas.drop(columns='Unnamed: 21', inplace=True)

Para estos datos vamos a enforcarnos solo en el año 2022, filtamos los Dataframe para el año.

In [39]:
# Filtramos las columnas que nos interesan, año y las renombramos para que tengan un nombre más amigable.

uso_bicicletas_2022 = uso_bicicletas[uso_bicicletas['Año'] == 2022].reset_index(drop=True)
uso_bicicletas_2022 = uso_bicicletas_2022.iloc[:, :9]
uso_bicicletas_2022.rename(columns={'Año':'anio',
                                    'DIA':'fecha',
                                    'Mes': 'mes',
                                    'Usos bicis abono anual día': 'uso_ab_anual_dia',
                                    'Usos bicis abono ocasional día':'uso_ab_ocas_dia',
                                    'Usos bicis total día':'uso_ab_total_dia',
                                    'Usos bicis abono anual acumulado mes':'uso_ab_anual_mes',
                                    'Usos bicis abono ocasional acumulado mes':'uso_ab_ocas_mes',
                                    'Usos bicis total acumulado mes':'uso_ab_total_mes'}, inplace=True)

Una vez unificadas la fuentes vamos a evaluar el tipo de dato y si tienen nulos o no.

In [40]:
# No tienen nulos

uso_bicicletas_2022.isnull().any()

anio                False
mes                 False
fecha               False
uso_ab_anual_dia    False
uso_ab_ocas_dia     False
uso_ab_total_dia    False
uso_ab_anual_mes    False
uso_ab_ocas_mes     False
uso_ab_total_mes    False
dtype: bool

In [41]:
''' 
Para nuestra analítica creamos columnas de fechas para poder trabajabar sobre todo la estacionalidad.
- Día de la semana: El número de día sabiendo que 0 es domingo y 6 es sabado.
- Fin de Semana: Variable binaria que indica si el día es sabado o domingo.

'''

uso_bicicletas_2022['dia_semana'] = [int(str(datetime.strftime(i, '%w'))) for i in uso_bicicletas_2022['fecha']]
uso_bicicletas_2022['fin_semana'] = [1 if i in [0, 6] else 0 for i in uso_bicicletas_2022['dia_semana']]

In [42]:
# Reordenamos la columnas

uso_bicicletas_2022 = uso_bicicletas_2022.loc[:, ['anio', 
                                                  'mes', 
                                                  'fecha', 
                                                  'dia_semana',
                                                  'fin_semana',
                                                  'uso_ab_anual_dia', 
                                                  'uso_ab_ocas_dia',
                                                  'uso_ab_total_dia', 
                                                  'uso_ab_anual_mes', 
                                                  'uso_ab_ocas_mes',
                                                  'uso_ab_total_mes']]

In [43]:
# Convertimos en enteros las variables desde la columna 5 al final.

for i in uso_bicicletas_2022.iloc[:, 5:]:
    uso_bicicletas_2022[i] = [int(x) for x in uso_bicicletas_2022[i]]

In [44]:
# Cambios en tipo de variable

uso_bicicletas_2022.dtypes

anio                         int64
mes                          int64
fecha               datetime64[ns]
dia_semana                   int64
fin_semana                   int64
uso_ab_anual_dia             int64
uso_ab_ocas_dia              int64
uso_ab_total_dia             int64
uso_ab_anual_mes             int64
uso_ab_ocas_mes              int64
uso_ab_total_mes             int64
dtype: object

In [45]:
uso_bicicletas_2022.head()

,anio,mes,fecha,dia_semana,fin_semana,uso_ab_anual_dia,uso_ab_ocas_dia,uso_ab_total_dia,uso_ab_anual_mes,uso_ab_ocas_mes,uso_ab_total_mes
0,2022,1,2022-01-01,6,1,3061,69,3130,3061,69,3130
1,2022,1,2022-01-02,0,1,3824,42,3866,6885,111,6996
2,2022,1,2022-01-03,1,0,5600,20,5620,12485,131,12616
3,2022,1,2022-01-04,2,0,5054,14,5068,17539,145,17684
4,2022,1,2022-01-05,3,0,3231,10,3241,20770,155,20925


In [46]:
# Base de datos tratado de uso abonos en el 2022.

uso_bicicletas_2022.to_csv('../data/Datos Tratados/uso_bici_resumen_2022.csv', index=False)

# URL 2: BiciMAD. Datos de la situación de estaciones bicimad por día y hora

In [47]:
# Funcion para leer varios ficheros de json de un directorio y convierte en dataframe.

sit_estaciones = u.call_situacion_estaciones()

In [48]:
# Nos quedamos con las columnas que nos interesa y las renombramos.

sit_estaciones = sit_estaciones.loc[:,['_id', 'id', 'number', 'name', 'activate', 'light', 'total_bases', 'free_bases', 'dock_bikes', 'reservations_count']]

sit_estaciones.rename(columns={'_id':'fecha_hora',
                               'id':'id_estacion',
                               'number':'cod_estacion',
                               'name':'des_estacion'}, inplace=True)

Creamos columnas de fechas para nuestro análisis.

In [49]:
''' 
Para nuestra analítica creamos columnas de fechas para poder trabajabar sobre todo la estacionalidad.
- fecha hora: Parseamos la columna para que nos identique como columna de fecha.
- fecha: Columnas solo de fecha.
- Mes: Columna de mes.
- Hora minuto: Columna de horas y minutos.
- Hora: Columna de hora formato 24 pero en int.
- Día de la semana: El número de día sabiendo que 0 es domingo y 6 es sabado.
- Fin de Semana: Variable binaria que indica si el día es sabado o domingo.

'''

sit_estaciones['fecha_hora'] = [datetime.strptime(i, '%Y-%m-%dT%H:%M:%S.%f') for i in sit_estaciones['fecha_hora']]
sit_estaciones['fecha'] = [datetime.strptime(datetime.strftime(i, '%Y-%m-%d'), '%Y-%m-%d')   for i in sit_estaciones['fecha_hora']]
sit_estaciones['mes'] = [int(str(datetime.strftime(i, '%m')))  for i in sit_estaciones['fecha_hora']]
sit_estaciones['hora_minuto'] = [datetime.strftime(i, '%H:%M') for i in sit_estaciones['fecha_hora']]
sit_estaciones['hora'] = [datetime.strftime(i, '%H') for i in sit_estaciones['fecha_hora']]
sit_estaciones['dia_semana'] = [int(str(datetime.strftime(i, '%w'))) for i in sit_estaciones['fecha_hora']]
sit_estaciones['fin_semana'] = [1 if i in [0, 6] else 0 for i in sit_estaciones['dia_semana']]

sit_estaciones = sit_estaciones.loc[:, ['fecha_hora', 
                                        'fecha',
                                        'mes',
                                        'hora_minuto',
                                        'hora',
                                        'dia_semana',
                                        'fin_semana',
                                        'id_estacion', 
                                        'cod_estacion', 
                                        'des_estacion', 
                                        'activate',
                                        'light',
                                        'total_bases',
                                        'free_bases',
                                        'dock_bikes',
                                        'reservations_count']]

In [50]:
#Nos quitamos los puntos de inactividad de las estaciones

sit_estaciones = sit_estaciones[(sit_estaciones['light']!=3) | (sit_estaciones['activate']!=0)] 

In [51]:
# Formatemos la columna del código de estación para que esté igual que el maestro de estaciones.

sit_estaciones['cod_estacion'] = [x.lstrip('0').replace(' ', '') for x in sit_estaciones['cod_estacion']]

In [52]:
sit_estaciones.head()

,fecha_hora,fecha,mes,hora_minuto,hora,dia_semana,fin_semana,id_estacion,cod_estacion,des_estacion,activate,light,total_bases,free_bases,dock_bikes,reservations_count
0,2022-01-01 00:13:20.603583,2022-01-01,1,00:13,00,6,1,1,1a,Puerta del Sol A,1,3,30,0,0,0
1,2022-01-01 00:13:20.603583,2022-01-01,1,00:13,00,6,1,2,1b,Puerta del Sol B,1,3,30,0,0,0
2,2022-01-01 00:13:20.603583,2022-01-01,1,00:13,00,6,1,3,2,Miguel Moya,1,0,24,16,7,0
3,2022-01-01 00:13:20.603583,2022-01-01,1,00:13,00,6,1,4,3,Plaza Conde Suchil,1,1,18,1,14,0
4,2022-01-01 00:13:20.603583,2022-01-01,1,00:13,00,6,1,5,4,Malasaña,1,1,24,2,17,0


In [53]:
# Base de datos tratado de Situaciones de las estaciones en el 2022.

sit_estaciones.to_csv('../data/Datos Tratados/sit_estaciones_2022.csv', index=False)

# URL 3: BiciMAD. Datos de los itinerarios de las bicicletas eléctricas

In [54]:
# Funcion para leer varios ficheros de csv de un directorio y convierte en dataframe.

intinerarios_bicicletas_2022 = u.call_uso_bicicletas()

In [55]:
''' 
Limpieza de las columnas de latitud y logitud ya que tenia datos con este formato "{'type': 'Point', 'coordinates': [-3.6714166, 40.4318611]}"
'''

localizaciones = ['geolocation_unlock', 'geolocation_lock']

for campo in localizaciones:
    subindice = campo.split('_')[1]
    lon_list = []
    lat_list = []
    for i in intinerarios_bicicletas_2022.loc[:, campo]:
        valor1 = i.split("'coordinates': [")[1]
        valor2 = valor1.replace(']}', '')
        lon, lat = valor2.split(',')
        lon_list.append(lon)
        lat_list.append(lat)
    campolon = 'lon'+'_'+ subindice
    campolat = 'lat'+'_'+ subindice
    intinerarios_bicicletas_2022[campolon] = lon_list
    intinerarios_bicicletas_2022[campolat] = lat_list

Reordenamos las columnas y nos quedamos con las importantes.

In [56]:
intinerarios_bicicletas_2022.drop(columns=['geolocation_unlock', 
                                           'geolocation_lock', 
                                           'fleet', 
                                           'address_unlock', 
                                           'locktype', 
                                           'unlocktype', 
                                           'idBike',
                                           'idTrip'], axis=1)

intinerarios_bicicletas_2022 = intinerarios_bicicletas_2022.loc[:, ['fecha', 
                                                                    'trip_minutes',
                                                                    'unlock_date',
                                                                    'lon_unlock',
                                                                    'lat_unlock',
                                                                    'station_unlock',
                                                                    'dock_unlock',
                                                                    'unlock_station_name',
                                                                    'lock_date',
                                                                    'lon_lock',
                                                                    'lat_lock',
                                                                    'station_lock',
                                                                    'dock_lock',
                                                                    'lock_station_name']]

In [57]:
# Nos quitamos los intinerarios cuyos valores de lat y lon estan vacios y codigos de estaciones vacios.

intinerarios_bicicletas_2022.drop(axis=0, index=intinerarios_bicicletas_2022[intinerarios_bicicletas_2022['lon_lock']=="'NaN'"].index, inplace=True)
intinerarios_bicicletas_2022.drop(axis=0, index=intinerarios_bicicletas_2022[intinerarios_bicicletas_2022['station_lock'].isnull()].index, inplace=True)

Creamos fechas para nuestro análisis.

In [58]:
''' 
Para nuestra analítica formateamos la columnas de fechas, latitud y logitud.

'''

intinerarios_bicicletas_2022['fecha'] = [datetime.strptime(i, '%Y-%m-%d')   for i in intinerarios_bicicletas_2022['fecha']]
intinerarios_bicicletas_2022['unlock_date'] = [datetime.strptime(i, '%Y-%m-%dT%H:%M:%S')   for i in intinerarios_bicicletas_2022['unlock_date']]
intinerarios_bicicletas_2022['lock_date'] = [datetime.strptime(i, '%Y-%m-%dT%H:%M:%S')   for i in intinerarios_bicicletas_2022['lock_date']]
intinerarios_bicicletas_2022['lon_unlock'] = [float(i) for i in intinerarios_bicicletas_2022['lon_unlock']]
intinerarios_bicicletas_2022['lat_unlock'] = [float(i) for i in intinerarios_bicicletas_2022['lat_unlock']]
intinerarios_bicicletas_2022['lon_lock'] = [float(i) for i in intinerarios_bicicletas_2022['lon_lock']]
intinerarios_bicicletas_2022['lat_lock'] = [float(i) for i in intinerarios_bicicletas_2022['lat_lock']]
intinerarios_bicicletas_2022['station_lock'] = [str(int(i)) for i in intinerarios_bicicletas_2022['station_lock']]

In [59]:
intinerarios_bicicletas_2022.head()

,fecha,trip_minutes,unlock_date,lon_unlock,lat_unlock,station_unlock,dock_unlock,unlock_station_name,lock_date,lon_lock,lat_lock,station_lock,dock_lock,lock_station_name
0,2022-01-01,16.28,2022-01-01 00:02:20,-3.671417,40.431861,200,3.0,Avenida de los Toreros,2022-01-01 00:18:37,-3.688398,40.419752,64,4.0,Plaza de la Independencia
1,2022-01-01,7.10,2022-01-01 00:07:53,-3.689419,40.416683,69,5.0,Antonio Maura,2022-01-01 00:14:59,-3.699346,40.430952,169,17.0,Manuel Silvela
2,2022-01-01,0.48,2022-01-01 00:09:21,-3.688822,40.409808,73,21.0,Puerta del Ángel Caído,2022-01-01 00:09:50,-3.688822,40.409808,73,21.0,Puerta del Ángel Caído
3,2022-01-01,6.62,2022-01-01 00:09:52,-3.665306,40.426000,192,22.0,Marqués de Zafra,2022-01-01 00:16:29,-3.665778,40.418667,190,17.0,Parque Roma
4,2022-01-01,8.07,2022-01-01 00:09:57,-3.698306,40.396222,183,3.0,Jaime el Conquistador,2022-01-01 00:18:01,-3.702502,40.410156,47,24.0,Jesús y María


In [60]:
# Base de datos tratado de intinerarios de la bicicletas en el 2022.

intinerarios_bicicletas_2022.to_csv('../data/Datos Tratados/intinerarios_bicicletas_2022.csv', index=False)

## URL 4: BiciMAD. Bases del servicio público de bicicleta eléctrica de la ciudad de Madrid

In [61]:
# Lectura de csv bases de las bicicletas de bicimad ultima actualizacion del 2020

mapa_bases = pd.read_csv('../data/Datos Originales/bases_bicimad.csv', sep=';')

In [62]:
# Limpieza de columnas y orden

mapa_bases['Fecha de Alta'] = [u.convertir_fecha(x) for x in mapa_bases['Fecha de Alta']]
mapa_bases.drop(['Gis_X', 'Gis_Y', 'Tipo de Reserva', 'Direccion'], axis=1, inplace=True)
mapa_bases.sort_values(by='Fecha de Alta', inplace=True, ignore_index=True)

# Columna de cod estacion viene con un formato diferente a las otras fuentes por lo que la formateamos para que quede igual

mapa_bases['Número'] = [x.lstrip('0').replace(' ', '') for x in mapa_bases['Número']]

In [63]:
# Existe registros duplicados de las estaciones ya que han tenido ampliaciones, por lo que nos quedamos con el registro ultimo que han tenido

for i in mapa_bases[mapa_bases['Número'].str.contains('ampliacion')].loc[:, 'Número'].values:
    valor = i.split('ampliacion')
    mapa_bases.drop(axis=0, index=mapa_bases[mapa_bases['Número'] == valor[0]].index, inplace=True)

mapa_bases.reset_index(inplace=True, drop=True)

mapa_bases['Número'] = [x.replace('ampliacion', '') for x in mapa_bases['Número']]
mapa_bases['Número'] = [x.replace('ampliación', '') for x in mapa_bases['Número']]

In [64]:
# Renombrar las variables

mapa_bases.rename(columns={'Número':'cod_estacion',
                           'Fecha de Alta':'fecha_alta',
                           'Distrito':'distrito',
                           'Barrio':'barrio',
                           'Calle':'calle',
                           'Nº Finca':'numero',
                           'Número de Plazas':'plazas',
                           'Longitud':'longitud',
                           'Latitud':'latitud'}, inplace=True)

Detectamos una base duplicada que es la 140, al parecer era una ampliación pero no tenia registrado como apliación por lo que nos quitamos el registro anterior.

In [65]:
# Eliminamos registro duplicado

mapa_bases.drop(axis=0, index=mapa_bases[(mapa_bases['cod_estacion']=='140')&(mapa_bases['plazas']==3)].index, inplace=True)

In [66]:
mapa_bases.head(1)

,cod_estacion,fecha_alta,distrito,barrio,calle,numero,plazas,longitud,latitud
0,106a,2014-05-23,04 SALAMANCA,04-01 RECOLETOS,"SERRANO, CALLE, DE",32,18,-3.687887,40.424864


In [67]:
# Base de datos tratado de mapas de las estaciones.

mapa_bases.to_csv('../data/Datos Tratados/mapa_bases_bicicletas.csv', index=False)

## URL 5: BiciMAD. Disponibilidad de unidades (bicicletas) del servicio público de bicicleta eléctrica

In [68]:
# Lectura de csv bases de las bicicletas de bicimad ultima actualizacion del 2020

dispo_bicis = pd.read_csv('../data/Datos Originales/bici_disponibilidad20221031.csv', sep=';')

In [69]:
# Quitamos registros nulos

dispo_bicis.dropna(how='all', inplace=True)

In [70]:
# Parse de fecha

dispo_bicis['DIA'] = [datetime.strptime(i, '%d/%m/%Y') for i in dispo_bicis['DIA']]

In [71]:
# Formateo de variables numericas y cambio de tipo

dispo_bicis['HORAS_TOTALES_USOS_BICICLETAS'] = [float(i.replace('.', '').replace(',', '.')) for i in dispo_bicis['HORAS_TOTALES_USOS_BICICLETAS']]
dispo_bicis['HORAS_TOTALES_DISPONIBILIDAD_BICICLETAS_EN _ANCLAJES'] = [float(i.replace('.', '').replace(',', '.')) for i in dispo_bicis['HORAS_TOTALES_DISPONIBILIDAD_BICICLETAS_EN _ANCLAJES']]
dispo_bicis['TOTAL_HORAS_SERVICIO_BICICLETAS'] = [float(i.replace('.', '').replace(',', '.')) for i in dispo_bicis['TOTAL_HORAS_SERVICIO_BICICLETAS']]
dispo_bicis['MEDIA_BICICLETAS_DISPONIBLES'] = [float(i.replace('.', '').replace(',', '.')) for i in dispo_bicis['MEDIA_BICICLETAS_DISPONIBLES']]
dispo_bicis['USOS_ABONADO_ANUAL'] = [int(str(i).replace('.', '')) for i in dispo_bicis['USOS_ABONADO_ANUAL']]
dispo_bicis['USOS_ABONADO_OCASIONAL'] = [int(str(i).replace('.', '')) for i in dispo_bicis['USOS_ABONADO_OCASIONAL']]
dispo_bicis['TOTAL_USOS'] = [int(str(i).replace('.', '')) for i in dispo_bicis['TOTAL_USOS']]

In [72]:
# Renombrar las variables

dispo_bicis.rename(columns={'DIA':'fecha',
                            'HORAS_TOTALES_USOS_BICICLETAS':'horas_uso_bicis',
                            'HORAS_TOTALES_DISPONIBILIDAD_BICICLETAS_EN _ANCLAJES':'horas_dispo_bicis_anclaje',
                            'TOTAL_HORAS_SERVICIO_BICICLETAS':'horas_total_serv_bicis',
                            'MEDIA_BICICLETAS_DISPONIBLES':'media_dispo_bicis',
                            'USOS_ABONADO_ANUAL':'uso_ab_anual',
                            'USOS_ABONADO_OCASIONAL':'uso_ab_ocas',
                            'TOTAL_USOS':'total_uso'
                            }, inplace=True)

Variables de fechas para el análisis.

In [73]:
dispo_bicis['mes'] = [int(str(datetime.strftime(i, '%m')))  for i in dispo_bicis['fecha']]
dispo_bicis['dia_semana'] = [int(str(datetime.strftime(i, '%w'))) for i in dispo_bicis['fecha']]
dispo_bicis['fin_semana'] = [1 if i in [0, 6] else 0 for i in dispo_bicis['dia_semana']]

In [74]:
# Reordenar variables

ispo_bicis = dispo_bicis.loc[:, ['fecha',
                                  'mes',
                                  'dia_semana',
                                  'fin_semana',
                                  'horas_uso_bicis',
                                  'horas_dispo_bicis_anclaje',
                                  'horas_total_serv_bicis',
                                  'media_dispo_bicis',
                                  'uso_ab_anual',
                                  'uso_ab_ocas',
                                  'total_uso']]

In [75]:
dispo_bicis.head()

,fecha,horas_uso_bicis,horas_dispo_bicis_anclaje,horas_total_serv_bicis,media_dispo_bicis,uso_ab_anual,uso_ab_ocas,total_uso,mes,dia_semana,fin_semana
0,2015-08-10,3991.13,19894.18,23885.31,995.22,7986,1890,8175,8,1,0
1,2015-08-11,4364.40,19817.30,24181.70,1007.57,875,1910,8941,8,2,0
2,2015-08-12,4633.72,19725.32,24359.04,1014.96,9132,2510,9383,8,3,0
3,2015-08-13,4498.85,19530.97,24029.82,1001.24,8974,2360,921,8,4,0
4,2015-08-14,4309.22,22770.43,27079.65,1128.32,8165,2310,8396,8,5,0


In [76]:
# Base de datos tratado de disposición de bicletas resumen por día.

dispo_bicis.to_csv('../data/Datos Tratados/dispo_bicis_2022.csv', index=False)

## URL 6: Aforo de Personas y Bicicletas

### Bicicletas

In [77]:
# Lectura de csv bases uso de bicicletas 

aforo_bicicletas_2022 = pd.read_csv('../data/Datos Originales/Aforo Personas y Bicicletas/BICICLETAS_2022.csv', sep=';')
aforo_bicicletas_2023 = pd.read_csv('../data/Datos Originales/Aforo Personas y Bicicletas/BICICLETAS_2023.csv', sep=';')

In [78]:
# Quitamos las variables que no necesitamos

aforo_bicicletas_2022.drop(axis=1,
                         columns=['observaciones_direccion',
                                  'identificador',
                                  'device_id',
                                  'NÃºmero_distrito',
                                  'distrito',
                                  'hora'], inplace=True)

aforo_bicicletas_2023.drop(axis=1,
                         columns=['observaciones_direccion',
                                  'identificador',
                                  'device_id',
                                  'NÃºmero_distrito',
                                  'distrito',
                                  'hora'], inplace=True)

Formateo de variables de fecha.

In [79]:
#2022 
aforo_bicicletas_2022['fecha_hora'] = [datetime.strptime(i, '%d/%m/%Y %H:%M') for i in aforo_bicicletas_2022['fecha']]
aforo_bicicletas_2022['fecha'] = [datetime.strftime(i, '%Y-%m-%d')  for i in aforo_bicicletas_2022['fecha_hora']]
aforo_bicicletas_2022['mes'] = [int(str(datetime.strftime(i, '%m')))  for i in aforo_bicicletas_2022['fecha_hora']]
aforo_bicicletas_2022['hora_min'] = [datetime.strftime(i, '%H:%M') for i in aforo_bicicletas_2022['fecha_hora']]
aforo_bicicletas_2022['dia_semana'] = [int(str(datetime.strftime(i, '%w'))) for i in aforo_bicicletas_2022['fecha_hora']]
aforo_bicicletas_2022['fin_semana'] = [1 if i in [0, 6] else 0 for i in aforo_bicicletas_2022['dia_semana']]
aforo_bicicletas_2022['latitude'] = [u.limpiar_coordenadas(x) for x in aforo_bicicletas_2022['latitude']]    
aforo_bicicletas_2022['longitude'] = [u.limpiar_coordenadas(x) for x in aforo_bicicletas_2022['longitude']]    
aforo_bicicletas_2022['hora'] = [int(datetime.strftime(i, '%H')) for i in aforo_bicicletas_2022['fecha_hora']]

#2024
aforo_bicicletas_2023['fecha_hora'] = [datetime.strptime(i, '%d/%m/%Y %H:%M') for i in aforo_bicicletas_2023['fecha']]
aforo_bicicletas_2023['fecha'] = [datetime.strftime(i, '%Y-%m-%d')  for i in aforo_bicicletas_2023['fecha_hora']]
aforo_bicicletas_2023['mes'] = [int(str(datetime.strftime(i, '%m')))  for i in aforo_bicicletas_2023['fecha_hora']]
aforo_bicicletas_2023['hora'] = [datetime.strftime(i, '%H:%M') for i in aforo_bicicletas_2023['fecha_hora']]
aforo_bicicletas_2023['dia_semana'] = [int(str(datetime.strftime(i, '%w'))) for i in aforo_bicicletas_2023['fecha_hora']]
aforo_bicicletas_2023['fin_semana'] = [1 if i in [0, 6] else 0 for i in aforo_bicicletas_2023['dia_semana']]
aforo_bicicletas_2023['latitude'] = [u.limpiar_coordenadas(x) for x in aforo_bicicletas_2023['latitude']]    
aforo_bicicletas_2023['longitude'] = [u.limpiar_coordenadas(x) for x in aforo_bicicletas_2023['longitude']]  

In [80]:
# Reordenar variables

aforo_bicicletas_2022 = aforo_bicicletas_2022.loc[:, ['fecha_hora', 
                                                      'fecha', 
                                                      'mes', 
                                                      'hora_min',
                                                      'hora', 
                                                      'dia_semana', 
                                                      'fin_semana',                            
                                                      'bicicletas', 
                                                      'direccion', 
                                                      'latitude', 
                                                      'longitude']]

aforo_bicicletas_2023 = aforo_bicicletas_2023.loc[:, ['fecha_hora', 
                                                      'fecha', 
                                                      'mes', 
                                                      'hora', 
                                                      'dia_semana', 
                                                      'fin_semana',                            
                                                      'bicicletas', 
                                                      'direccion', 
                                                      'latitude', 
                                                      'longitude']]

In [81]:
aforo_bicicletas_2022.isnull().any()

fecha_hora    False
fecha         False
mes           False
hora_min      False
hora          False
dia_semana    False
fin_semana    False
bicicletas    False
direccion     False
latitude      False
longitude     False
dtype: bool

In [82]:
# Aforo de bicicletas 2023 tiene nulos por lo que quitamos los registros nulos

aforo_bicicletas_2023.isnull().any()

fecha_hora    False
fecha         False
mes           False
hora          False
dia_semana    False
fin_semana    False
bicicletas     True
direccion     False
latitude      False
longitude     False
dtype: bool

In [83]:
aforo_bicicletas_2023.dropna(inplace=True)

In [84]:
# Reemplazar los valores de letras mal codificadas

aforo_bicicletas_2022['direccion'] = aforo_bicicletas_2022['direccion'].apply(u.corregir_letras)

aforo_bicicletas_2023['direccion'] = aforo_bicicletas_2023['direccion'].apply(u.corregir_letras)

In [85]:
# Base de datos tratado de aforo de bicletas resumen por día.

aforo_bicicletas_2022.to_csv('../data/Datos Tratados/aforo_bicicletas_2022.csv', index=False)
aforo_bicicletas_2023.to_csv('../data/Datos Tratados/aforo_bicicletas_2023.csv', index=False)

### Personas

In [86]:
# Lectura de csv de aforo de las personas

aforo_personas_2022 = pd.read_csv('../data/Datos Originales/Aforo Personas y Bicicletas/PEATONES_2022.csv', sep=';')
aforo_personas_2023 = pd.read_csv('../data/Datos Originales/Aforo Personas y Bicicletas/PEATONES_2023.csv', sep=';')

In [87]:
# nos quedamos con las variables necesarias

aforo_personas_2022.drop(axis=1,
                         columns=['observaciones_direccion',
                                  'identificador',
                                  'device_id',
                                  'NÃºmero_distrito',
                                  'distrito',
                                  'hora'], inplace=True)

aforo_personas_2023.drop(axis=1,
                         columns=['observaciones_direccion',
                                  'identificador',
                                  'device_id',
                                  'NÃºmero_distrito',
                                  'distrito',
                                  'hora'], inplace=True)

Formateo de variables de fecha.

In [88]:
#2022
aforo_personas_2022['fecha_hora'] = [datetime.strptime(i, '%d/%m/%Y %H:%M') for i in aforo_personas_2022['fecha']]
aforo_personas_2022['fecha'] = [datetime.strftime(i, '%Y-%m-%d')  for i in aforo_personas_2022['fecha_hora']]
aforo_personas_2022['mes'] = [int(str(datetime.strftime(i, '%m')))  for i in aforo_personas_2022['fecha_hora']]
aforo_personas_2022['hora_min'] = [datetime.strftime(i, '%H:%M') for i in aforo_personas_2022['fecha_hora']]
aforo_personas_2022['dia_semana'] = [int(str(datetime.strftime(i, '%w'))) for i in aforo_personas_2022['fecha_hora']]
aforo_personas_2022['fin_semana'] = [1 if i in [0, 6] else 0 for i in aforo_personas_2022['dia_semana']]
aforo_personas_2022['latitude'] = [u.limpiar_coordenadas(x) for x in aforo_personas_2022['latitude']]    
aforo_personas_2022['longitude'] = [u.limpiar_coordenadas(x) for x in aforo_personas_2022['longitude']] 
aforo_personas_2022['hora'] = [int(datetime.strftime(i, '%H')) for i in aforo_personas_2022['fecha_hora']]

#2024
aforo_personas_2023['fecha_hora'] = [datetime.strptime(i, '%d/%m/%Y %H:%M') for i in aforo_personas_2023['fecha']]
aforo_personas_2023['fecha'] = [datetime.strftime(i, '%Y-%m-%d')  for i in aforo_personas_2023['fecha_hora']]
aforo_personas_2023['mes'] = [int(str(datetime.strftime(i, '%m')))  for i in aforo_personas_2023['fecha_hora']]
aforo_personas_2023['hora'] = [datetime.strftime(i, '%H:%M') for i in aforo_personas_2023['fecha_hora']]
aforo_personas_2023['dia_semana'] = [int(str(datetime.strftime(i, '%w'))) for i in aforo_personas_2023['fecha_hora']]
aforo_personas_2023['fin_semana'] = [1 if i in [0, 6] else 0 for i in aforo_personas_2023['dia_semana']]
aforo_personas_2023['latitude'] = [u.limpiar_coordenadas(x) for x in aforo_personas_2023['latitude']]    
aforo_personas_2023['longitude'] = [u.limpiar_coordenadas(x) for x in aforo_personas_2023['longitude']]  

In [89]:
# Reordenar varaiables

aforo_personas_2022 = aforo_personas_2022.loc[:, ['fecha_hora', 
                                                      'fecha', 
                                                      'mes',
                                                      'hora_min', 
                                                      'hora', 
                                                      'dia_semana', 
                                                      'fin_semana',                            
                                                      'peatones', 
                                                      'direccion', 
                                                      'latitude', 
                                                      'longitude']]

aforo_personas_2023 = aforo_personas_2023.loc[:, ['fecha_hora', 
                                                      'fecha', 
                                                      'mes', 
                                                      'hora', 
                                                      'dia_semana', 
                                                      'fin_semana',                            
                                                      'peatones', 
                                                      'direccion', 
                                                      'latitude', 
                                                      'longitude']]

In [90]:
aforo_personas_2022.isnull().any()

fecha_hora    False
fecha         False
mes           False
hora_min      False
hora          False
dia_semana    False
fin_semana    False
peatones      False
direccion     False
latitude      False
longitude     False
dtype: bool

In [91]:
# Aforo de personas 2023 tiene nulos por lo que quitamos los registros nulos

aforo_personas_2023.isnull().any()

fecha_hora    False
fecha         False
mes           False
hora          False
dia_semana    False
fin_semana    False
peatones       True
direccion     False
latitude      False
longitude     False
dtype: bool

In [92]:
aforo_personas_2023.dropna(inplace=True)

In [93]:
# Reemplazar los valores de letras mal codificadas

aforo_personas_2022['direccion'] = aforo_personas_2022['direccion'].apply(u.corregir_letras)

aforo_personas_2023['direccion'] = aforo_personas_2023['direccion'].apply(u.corregir_letras)

In [94]:
aforo_personas_2022.head()

,fecha_hora,fecha,mes,hora_min,hora,dia_semana,fin_semana,peatones,direccion,latitude,longitude
0,2022-09-01 00:00:00,2022-09-01,9,00:00,0,4,0,136,"Calle Fuencarral 22, 28004 Madrid",40.422009,-3.700892
1,2022-09-01 01:00:00,2022-09-01,9,01:00,1,4,0,129,"Calle Fuencarral 22, 28004 Madrid",40.422009,-3.700892
2,2022-09-01 02:00:00,2022-09-01,9,02:00,2,4,0,138,"Calle Fuencarral 22, 28004 Madrid",40.422009,-3.700892
3,2022-09-01 03:00:00,2022-09-01,9,03:00,3,4,0,244,"Calle Fuencarral 22, 28004 Madrid",40.422009,-3.700892
4,2022-09-01 04:00:00,2022-09-01,9,04:00,4,4,0,527,"Calle Fuencarral 22, 28004 Madrid",40.422009,-3.700892


In [95]:
aforo_personas_2023.head()

,fecha_hora,fecha,mes,hora,dia_semana,fin_semana,peatones,direccion,latitude,longitude
0,2023-07-16 01:00:00,2023-07-16,7,01:00,0,1,984.0,Calle Arenal esquina San Martín,40.417386,-3.707141
1,2023-07-16 02:00:00,2023-07-16,7,02:00,0,1,601.0,Calle Arenal esquina San Martín,40.417386,-3.707141
2,2023-07-16 03:00:00,2023-07-16,7,03:00,0,1,399.0,Calle Arenal esquina San Martín,40.417386,-3.707141
3,2023-07-16 04:00:00,2023-07-16,7,04:00,0,1,300.0,Calle Arenal esquina San Martín,40.417386,-3.707141
4,2023-07-16 05:00:00,2023-07-16,7,05:00,0,1,149.0,Calle Arenal esquina San Martín,40.417386,-3.707141


In [96]:
# Base de datos tratado de aforo de personas resumen por día.

aforo_personas_2022.to_csv('../data/Datos Tratados/aforo_personas_2022.csv', index=False)
aforo_personas_2023.to_csv('../data/Datos Tratados/aforo_personas_2023.csv', index=False)

## URL 7: Afluencia Turistas 2022

In [97]:
# Lectura de csv de turistas por mes en comunidad de Madrid

turistas_2022 = pd.read_csv('../data/Datos Originales/Flujo_Turistas_2022.csv', sep=';', encoding='latin-1')

In [98]:
# Nos quedamos con las variables que necesitamos y renombramos

turistas_2022.drop(axis=1,
                   columns=['Comunidades autónomas', 
                            'Tipo de dato'], inplace=True)

turistas_2022.rename(columns={'Total':'num_turistas',
                              'Periodo':'periodo'}, inplace=True)

In [99]:
# Formateo de fechas

turistas_2022['mes'] = [int(x.split('M')[1]) for x in turistas_2022['periodo']]
turistas_2022['periodo'] = turistas_2022['periodo'].str.replace('M', '')
turistas_2022['periodo'] = [datetime.strptime(i, '%Y%m').strftime('%Y-%m') for i in turistas_2022['periodo']]
turistas_2022['num_turistas'] = [int(float(i.replace('.', '').replace(',', '.'))) for i in turistas_2022['num_turistas']]

In [100]:
# Reordenar las varaibles

turistas_2022 = turistas_2022.loc[:, ['periodo',
                                      'mes',
                                      'num_turistas']]

In [101]:
# Base de datos tratado de turistas

turistas_2022.to_csv('../data/Datos Tratados/turistas_2022.csv', index=False)